# Data Preprocessing

In [4]:
from pyhealth.datasets import MIMIC3Dataset

dataset = MIMIC3Dataset(
    root='data/',
    tables=["DIAGNOSES_ICD","PROCEDURES_ICD"],
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})}
)


In [5]:
dataset.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PROCEDURES_ICD: 4.0711



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

In [6]:
# get patient dictionary
patient_dict = dataset.patients
print(list(patient_dict.keys())[:10])

['10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005', '10006']


In [7]:
# get the "10" patient
patient = patient_dict["10"]
patient.gender, patient.birth_datetime, patient.ethnicity, patient.death_datetime, patient.visits
print(patient.visits)

OrderedDict([('184167', Visit 184167 from patient 10 with 8 events from tables ['DIAGNOSES_ICD', 'PROCEDURES_ICD'])])


In [8]:
# get the visit list of this patient
visit_dict = patient.visits
print (list(visit_dict.keys()))

['184167']


In [9]:
# get the first visit
visit = visit_dict['184167']
visit.encounter_time, visit.available_tables, visit.num_events, visit.event_list_dict

(datetime.datetime(2103, 6, 28, 11, 36),
 ['DIAGNOSES_ICD', 'PROCEDURES_ICD'],
 8,
 {'DIAGNOSES_ICD': [Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
   Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
   Event with ICD9CM code V290 from table DIAGNOSES_ICD],
  'PROCEDURES_ICD': [Event with ICD9PROC code 9983 from table PROCEDURES_ICD,
   Event with ICD9PROC code 9915 from table PROCEDURES_ICD,
   Event with ICD9PROC code 966 from table PROCEDURES_ICD]})

In [10]:
visit.get_event_list('DIAGNOSES_ICD')

[Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
 Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
 Event with ICD9CM code V290 from table DIAGNOSES_ICD]

In [11]:
visit.get_code_list('DIAGNOSES_ICD')

['V3000', '7742', '76525', '76515', 'V290']

In [1]:
import pandas as pd

table = "NOTEEVENTS"

# read table
df = pd.read_csv(
    "data/NOTEEVENTS.csv",
    dtype={"SUBJECT_ID": str, "HADM_ID": str, "CATEGORY": str,"CHARTDATE": str, "TEXT": str},
    parse_dates=["CHARTDATE"],
    infer_datetime_format=True
)


C:\Users\giuse\AppData\Local\Temp\ipykernel_13236\2530625872.py:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [2]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [12]:
# filter for 'DISCHARGE SUMMARY' in the 'CATEGORY' column
df = df[df['CATEGORY'] == 'Discharge summary']
# drop records of the other patients
df = df[df["SUBJECT_ID"].isin(list(patient_dict.keys())[10000:10005])]
# drop rows with missing values
df = df.dropna(subset=["SUBJECT_ID", "HADM_ID", "TEXT"])
# group by patient and visit
group_df = df.groupby("SUBJECT_ID")

In [13]:
df.set_index("ROW_ID", inplace=True)

In [14]:
df

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
ROW_ID,,,,,,,,,,
844,19544,180092,2200-04-05,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2200-3-24**] ...
845,19544,163687,2201-02-19,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2201-1-29**] ...
17528,19548,193954,2170-05-19,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2170-5-10**] Discharge...
16687,19546,194718,2146-12-22,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2146-12-17**] Discha...
27275,19545,109452,2146-05-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2146-4-24**] Dischar...
43699,19547,106062,2151-06-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-5-10**] ...
43700,19547,127546,2152-02-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-2-1**] D...
43701,19547,138551,2152-04-02,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-3-28**] ...


In [16]:
# Define the ranges for subject IDs
ranges = [(10000, 20000), (20000, 30000), (30000, 40000)]

# Filter 5 patients from each range
selected_patients = []
for start, end in ranges:
    patients_in_range = df[(df['SUBJECT_ID'].astype(int) > start) & (df['SUBJECT_ID'].astype(int) <= end)]
    selected_patients.extend(patients_in_range.sample(5).to_dict('records'))

# Convert the list of dicts to a dataframe
selected_patients_df = pd.DataFrame(selected_patients)



In [17]:
selected_patients_list = selected_patients_df['SUBJECT_ID'].tolist()
selected_patients_list

['13631',
 '12285',
 '15923',
 '19593',
 '12140',
 '20629',
 '25333',
 '24444',
 '24398',
 '23460',
 '30225',
 '32247',
 '31526',
 '30299',
 '32040']

In [36]:
filtered_noteevents_df = df[df['SUBJECT_ID'].isin(selected_patients_list)]

In [37]:
filtered_noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT


In [ ]:
#unique_subject_ids = noteevents_df['SUBJECT_ID'].unique()[:10]
#filtered_noteevents_df = noteevents_df[noteevents_df['SUBJECT_ID'].isin(unique_subject_ids)]

In [15]:
from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT

c:\Users\giuse\Desktop\MSc-Thesis-Project\proj_venv\lib\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [16]:
DATA_DIR = "./data_umls/"
vocab_path = DATA_DIR + "vocab.dat"
cdb_path = DATA_DIR + "cdb.dat"

In [17]:
# Create and load the CDB (Concept Database)
cdb = CDB.load(cdb_path)

# Create and load the Vocabulary
vocab = Vocab.load(vocab_path)

# Load model for MetaAnnotations (optional)
mc_status = MetaCAT.load("./data_umls/meta_Status/")

In [18]:
# Create CAT - the main class from medcat used for concept annotation
cat = CAT(cdb=cdb, config=cdb.config, vocab=vocab, meta_cats=[mc_status])

c:\Users\giuse\Desktop\MSc-Thesis-Project\proj_venv\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [19]:
# Detect only the following entity:
    # bpoc|T023|Body Part, Organ, or Organ Component
    # lbtr|T034|Laboratory or Test Result
    # patf|T046|Pathologic Function
    # dsyn|T047|Disease or Syndrome
    # mobd|T048|Mental or Behavioral Dysfunction
    # lbpr|T059|Laboratory Procedure
    # diap|T060|Diagnostic Procedure
    # topp|T061|Therapeutic or Preventive Procedure
    # phsu|T121|Pharmacologic Substance ---> SE LO AGGIUNGO MI METTE ANCHE ALTRA ROBA TIPO VITAMINE, ENZIMI, ETC.
    # sosy|T184|Sign or Symptom

type_ids_filter = ['T047','T048','T184']
cui_filters = set()
for type_ids in type_ids_filter:
    cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters

In [20]:
color_dict = {
    'Body Part, Organ, or Organ Component': '#FFCE80',  # Arancione pastello
    'Laboratory or Test Result': '#FFF9C4',             # Giallo chiaro
    'Disease or Syndrome': '#B5EAD7',                   # Verde acqua chiaro
    'Mental or Behavioral Dysfunction': '#F0B2FF',      # Viola pastello
    'Laboratory Procedure': '#D0D9FF',                  # Blu chiaro
    'Diagnostic Procedure': '#FFD9EC',                  # Rosa chiaro
    'Therapeutic or Preventive Procedure': '#C4FFFF',   # Azzurro chiaro
    'Pharmacologic Substance': '#FFDAB9',               # Pesca pastello
    'Sign or Symptom': '#FFC4F3',                       # Magenta chiaro
    'Pathologic Function': '#B5EAD7'                    # Verde acqua chiaro
}

In [23]:
entities_lemma = []
for row in df.index:
    ner = cat.get_entities(df['TEXT'][row])
    ner['row_id']=row
    ner['subject_id']=df['SUBJECT_ID'][row]
    ner['hadm_id']=df['HADM_ID'][row]
    entities_lemma.append(ner)


In [48]:
entities_lemma

[{'entities': {11: {'pretty_name': 'Dyspnea',
    'cui': 'C0013404',
    'type_ids': ['T184'],
    'types': ['Sign or Symptom'],
    'source_value': 'Dyspnea',
    'detected_name': 'dyspnea',
    'acc': 0.7845305305934716,
    'context_similarity': 0.7845305305934716,
    'start': 262,
    'end': 269,
    'icd10': [{'chapter': 'R06.0', 'name': 'Dyspnoea'},
     {'chapter': 'R06.00', 'name': 'Dyspnea, unspecified'},
     {'chapter': 'R06.02', 'name': 'Shortness of breath'}],
    'ontologies': [],
    'snomed': ['S-139192008',
     'S-139200001',
     'S-139201002',
     'S-139207003',
     'S-158371003',
     'S-158379001',
     'S-161937008',
     'S-161945003',
     'S-161946002',
     'S-161951008',
     'S-207057006',
     'S-207059009',
     'S-207062007',
     'S-230145002',
     'S-267036007',
     'S-308149007',
     'S-49233005'],
    'id': 11,
    'meta_anns': {'Status': {'value': 'Affirmed',
      'confidence': 0.9999895095825195,
      'name': 'Status'}}},
   13: {'pretty_na

In [44]:
from icdmappings import Mapper

mapper = Mapper()

# Example data structure for the dataframe
df_structure = {
    'ROW_ID': [],
    'SUBJECT_ID': [],
    'HADM_ID': [],
    'SEQ_NUM': [],
    'ICD10_CODE': [],
    'ICD9_CODE': []
}

# Process the data to extract needed information
for record in entities_lemma:
    row_id = record['row_id']
    subject_id = record['subject_id']
    hadm_id = record['hadm_id']
    entities = record['entities']
    seq_num = 1  # Reset sequence number for each record

    for entity_id, entity in sorted(entities.items()):
        # Read ICD-10 information, if available
        icd10_info = entity.get('icd10', [])
        
        # If ICD-10 information is not empty, append data to the structure
        if icd10_info:
            for icd10 in icd10_info:
                # icd9 to icd10
                # Remove the period from the ICD-10 code
                icd10code_cleaned = icd10['chapter'].replace('.', '')
                icd9 = mapper.map(icd10code_cleaned, source='icd10', target='icd9')
                if icd9:
                    df_structure['ROW_ID'].append(row_id)
                    df_structure['SUBJECT_ID'].append(subject_id)
                    df_structure['HADM_ID'].append(hadm_id)
                    df_structure['SEQ_NUM'].append(seq_num)
                    df_structure['ICD10_CODE'].append(icd10['chapter'])  # Using ICD-10 'chapter' as a placeholder for ICD-9 code
                    df_structure['ICD9_CODE'].append(icd9)  # Using ICD-10 'chapter' as a placeholder for ICD-9 code
                    seq_num += 1  # Increment SEQ_NUM only if icd10_info is not empty
                else:
                    break
            
# Create the dataframe
noteevents_df = pd.DataFrame(df_structure)

In [46]:
df_prova = noteevents_df[noteevents_df['SUBJECT_ID'].astype(int)==19544]

df_prova.head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD10_CODE,ICD9_CODE
0,844,19544,180092,1,R69,V419
1,844,19544,180092,2,I35.0,4241
2,844,19544,180092,3,Q25.3,74722
3,844,19544,180092,4,Q25.1,74710
4,844,19544,180092,5,I73.9,4439
5,844,19544,180092,6,G62.9,3579
6,844,19544,180092,7,H35.9,3629
7,844,19544,180092,8,D50.9,2809
8,844,19544,180092,9,I35.0,4241
9,844,19544,180092,10,Q25.3,74722


In [ ]:
extracted_data = []
# Extract 'pretty_name' and 'types' values from the dictionary
for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        if value['types'][0] == 'Pharmacologic Substance':
            extracted_data.append((value['source_value'], value['types'][0], value['icd10']))
        else:
            extracted_data.append((value['pretty_name'], value['types'][0], value['icd10']))

# Create a dataframe
df_extra = pd.DataFrame(extracted_data, columns=['Nome', 'Entità', 'ICD10'])


In [29]:
from spacy import displacy

doc = {"text": df['TEXT'][5315], "ents": [], "title": None}

for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        ent = {
            'start': value['start'],
            'end': value['end'],
            'label': value['types'][0]
        }
        doc['ents'].append(ent)

colors = color_dict
options = {"ents": list(colors.keys()), "colors": colors}

# Call spacy.displacy.render() to visualize the entities
displacy.render(doc, style='ent', options=options, jupyter=True, manual=True)

In [30]:
extracted_data

[('Lung diseases',
  'Disease or Syndrome',
  [{'chapter': 'J98.4', 'name': 'Other disorders of lung'}]),
 ('Peptic Ulcer',
  'Disease or Syndrome',
  [{'chapter': 'K27', 'name': 'Peptic ulcer, site unspecified'}]),
 ('Hypercholesterolemia', 'Disease or Syndrome', []),
 ('Gastroesophageal reflux disease',
  'Disease or Syndrome',
  [{'chapter': 'K21', 'name': 'Gastro-oesophageal reflux disease'},
   {'chapter': 'K21.9',
    'name': 'Gastro-esophageal reflux disease without esophagitis'}]),
 ('Chronic Obstructive Airway Disease',
  'Disease or Syndrome',
  [{'chapter': 'J44.9',
    'name': 'Chronic obstructive pulmonary disease, unspecified'}]),
 ('Pulmonary Emphysema',
  'Disease or Syndrome',
  [{'chapter': 'J43', 'name': 'Emphysema'},
   {'chapter': 'J43.9', 'name': 'Emphysema, unspecified'}]),
 ('Lung diseases',
  'Disease or Syndrome',
  [{'chapter': 'J98.4', 'name': 'Other disorders of lung'}]),
 ('Peptic Ulcer',
  'Disease or Syndrome',
  [{'chapter': 'K27', 'name': 'Peptic ulcer